In [1]:
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader

data_dir = 'flowers/'

transform = transforms.Compose([
    transforms.Resize((128, 128)),  # Изменение размера изображений
    transforms.ToTensor(),         # Конвертация в тензор
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # Нормализация
])

dataset = ImageFolder(root=data_dir, transform=transform)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

In [5]:
dataset.classes

['daisy', 'dandelion', 'rose', 'sunflower', 'tulip']

In [3]:
from sklearn.model_selection import train_test_split
train_data, val_data = train_test_split(dataset, test_size=0.2, random_state=42)

In [4]:
train_data

[(tensor([[[-0.8039, -0.7804, -0.7647,  ..., -0.8902, -0.8980, -0.8902],
           [-0.8431, -0.8196, -0.7882,  ..., -0.8980, -0.8902, -0.8745],
           [-0.8667, -0.8431, -0.8275,  ..., -0.8980, -0.8902, -0.8745],
           ...,
           [-0.8902, -0.8902, -0.8824,  ..., -0.9765, -0.9529, -0.9451],
           [-0.8902, -0.8824, -0.8745,  ..., -0.9765, -0.9686, -0.9451],
           [-0.8980, -0.8745, -0.8745,  ..., -0.9843, -0.9608, -0.9451]],
  
          [[-0.6941, -0.6706, -0.6549,  ..., -0.7647, -0.7490, -0.7412],
           [-0.7255, -0.7020, -0.6706,  ..., -0.7490, -0.7412, -0.7412],
           [-0.7569, -0.7255, -0.6941,  ..., -0.7412, -0.7412, -0.7412],
           ...,
           [-0.7804, -0.7725, -0.7647,  ..., -0.8824, -0.8667, -0.8588],
           [-0.7725, -0.7725, -0.7647,  ..., -0.8745, -0.8667, -0.8588],
           [-0.7569, -0.7647, -0.7647,  ..., -0.8745, -0.8588, -0.8510]],
  
          [[-0.9373, -0.9137, -0.8902,  ..., -0.9922, -0.9922, -0.9922],
           